In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path

# settings for making nice pdfs
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
# specify common string of the output folders (each folder contains results from independent runs, n>=3) 
base_folder_string = 'output_0601_tiling600_LonglivedLEFs_run'
folders = [foldername for foldername in os.listdir('Data/') if base_folder_string in foldername]

base_stochasticity = 1 # use to convert step to time

# loop through different folders
for m in range(len(folders)):
    # folder containing simulation results from each independent run
    data_dir = folders[m]
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    # parse file name
    # separation and processivity
    sep_proc_list = [ 'sep125_proc250', 'sep125_proc500', 'sep125_proc1000']
    # %long-lived LEFs
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    # processivity of long-lived LEFs
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [int(a) for a in superproc_list_sorted]
    # fold stabilization of LEFs at BE
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    # fold stabilization of LEF at DSB ends
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    # fold increase in loading probability at DSB
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [int(a) for a in superloading_list_sorted]
    # boundary strength
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        ctcf_bs_success =  np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        ctcf_bs_time0 =  np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))

    for i in range(len(superportion_list_sorted)):
        for j in range(len(sep_proc_list)):
            proc = float(sep_proc_list[j].split('_proc')[1])
            if proc == 62.5:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(proc*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]
            else:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(int(proc*20)) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]

            filename = 'Data/'+data_dir + '/stats_df_' + par_combo + '.csv'

            stats_df = pd.read_csv(filename)
            stats_df = stats_df.drop(columns='Unnamed: 0')
            stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
            sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
            if len(sep_proc_1st_pass)>0:
                ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 
            else:
                ctcf_bs_fp[m,i,j]=np.nan


            portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
            portion_df = portion_df.drop(columns='Unnamed: 0')
            portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
            ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]*100
            ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100



# Theory prediction from Mathematica Notebook LonglivedLEFs_20211004.nb, Prob
theory = np.asarray([[0.0484416, 0.0751836, 0.101215, 0.125764, 0.148581, 0.169657, \
0.189083, 0.206989, 0.223514, 0.238794, 0.252953, 0.266103, 0.278344, \
0.289764, 0.300443, 0.31045, 0.319844, 0.328681, 0.337009, 0.344871, \
0.352303],[0.113293, 0.157305, 0.197177, 0.232918, 0.264895, 0.293556, 0.319333, \
0.34261, 0.363718, 0.38294, 0.400513, 0.41664, 0.431491, 0.445214, \
0.457932, 0.469753, 0.48077, 0.491064, 0.500703, 0.50975, 0.518258],[0.204531, 0.261831, 0.310901, 0.353072, 0.389563, 0.421383, 0.449341, \
0.474086, 0.496133, 0.515897, 0.533715, 0.54986, 0.564557, 0.577996, \
0.59033, 0.601693, 0.612196, 0.621934, 0.630989, 0.63943, 0.647319]])*100

flist = [0., 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, \
0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]


colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))


for i in range(len(sep_proc_list)):
    axs[0].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.nanmean(ctcf_bs_success,axis=0)[:,i]),yerr =stats.sem(ctcf_bs_success,axis=0,nan_policy='omit')[:,i],fmt='s',markersize = 8, ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)

for i in range(len(sep_proc_list)):
    axs[0].plot(flist,theory[i],color= colors[i],linewidth=3,linestyle='dashed')

axs[0].set_ylim(0,100)
axs[0].set_ylabel('Synapsis Efficiency (%)')
axs[0].set_xlabel('% Long-lived LEFs')
axs[0].set_xticks(superportion_numerical_sorted[0:])
axs[0].tick_params(direction='out', length=8, width=2)

for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)
    
# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    
for i in range(len(sep_proc_list)):
    axs[1].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.mean(ctcf_bs_fp,axis=0)[:,i]),yerr =stats.sem(ctcf_bs_fp,axis=0)[:,i],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

axs[1].set_ylabel('Mean synapsis time (min)')
axs[1].set_xlabel('% Long-lived LEFs')
axs[1].tick_params(direction='out', length=8, width=2)
axs[1].set_xticks(superportion_numerical_sorted[0:])
axs[1].tick_params(direction='out', length=8, width=2)

for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+ sep_proc.split('_proc')[1] for sep_proc in sep_proc_list]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')



plt.savefig('Figures/'+'Longlived_LEFs.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()

In [ ]:
base_folder_string = 'output_0601_with residence_10 Mb apart_tiling600_2populationLEFs_GapMethod_withLonglivedLEFs_run'
folders = [foldername for foldername in os.listdir('Data/') if base_folder_string in foldername]

base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir = folders[m]

    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_proc_list = [ 'sep125_proc250', 'sep125_proc500', 'sep125_proc1000']
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [int(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [int(a) for a in superloading_list_sorted]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        ctcf_bs_success =  np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        ctcf_bs_time0 =  np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))

    for i in range(len(superportion_list_sorted)):
        for j in range(len(sep_proc_list)):
            proc = float(sep_proc_list[j].split('_proc')[1])
            if proc == 62.5:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(proc*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]
            else:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(int(proc*20)) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]

            filename = 'Data/'+ data_dir + '/stats_df_' + par_combo + '.csv'

            stats_df = pd.read_csv(filename)
            stats_df = stats_df.drop(columns='Unnamed: 0')
            stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
            sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
            if len(sep_proc_1st_pass)>0:
                ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 
            else:
                ctcf_bs_fp[m,i,j]=np.nan


            portion_df = pd.read_csv('Data/'+ data_dir + '/restrained_df_' + par_combo + '.csv')
            portion_df = portion_df.drop(columns='Unnamed: 0')
            portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
            ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]*100
            ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100

# Theory prediction from Mathematica Notebook LonglivedLEFs_20211004.nb, Probconstrained
theory = np.asarray([[0.707582, 0.741309, 0.767454, 0.78829, 0.805274, 0.819375, 0.831268, \
0.841431, 0.850217, 0.857887, 0.864642, 0.870637, 0.875995, 0.880812, \
0.885168, 0.889126, 0.892739, 0.896052, 0.8991, 0.901916, 0.904524],[0.822697, 0.843865, 0.859746, 0.872104, 0.882, 0.890108, 0.896878, \
0.902621, 0.907558, 0.911851, 0.915621, 0.918961, 0.921942, 0.924622, \
0.927045, 0.929249, 0.931262, 0.93311, 0.934813, 0.936389, 0.937851],[0.892005, 0.903991, 0.912889, 0.919777, 0.925283, 0.929796, 0.933572, \
0.936784, 0.939556, 0.941976, 0.944112, 0.946013, 0.947718, 0.949258, \
0.950658, 0.951938, 0.953113, 0.954196, 0.9552, 0.956133, 0.957003]])*100


flist = [0., 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, \
0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]


colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))


for i in range(len(sep_proc_list)):
    axs[0].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.nanmean(ctcf_bs_time0,axis=0)[:,i]),yerr =stats.sem(ctcf_bs_time0,axis=0,nan_policy='omit')[:,i],fmt='s',markersize = 8, ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)

for i in range(len(sep_proc_list)):
    axs[0].plot(flist,theory[i],color= colors[i],linewidth=3,linestyle='dashed')

axs[0].set_ylim(0,100)
axs[0].set_ylabel('% DSB constrained')
axs[0].set_xlabel('% Long-lived LEFs')
axs[0].set_xticks(superportion_numerical_sorted[0:])
axs[0].tick_params(direction='out', length=8, width=2)

for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)
    
# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    
for i in range(len(sep_proc_list)):
    axs[1].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.mean(ctcf_bs_fp,axis=0)[:,i]),yerr =stats.sem(ctcf_bs_fp,axis=0)[:,i],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

axs[1].set_ylabel('Mean synapsis time (min)')
axs[1].set_xlabel('% Long-lived LEFs')
axs[1].tick_params(direction='out', length=8, width=2)
axs[1].set_xticks(superportion_numerical_sorted[0:])
axs[1].tick_params(direction='out', length=8, width=2)

for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+ sep_proc.split('_proc')[1] for sep_proc in sep_proc_list]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')



plt.savefig('Figures/'+'Longlived_LEFs_Pconstrained.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()

In [ ]:
base_folder_string = 'output_0601_with residence_10 Mb apart_tiling600_2populationLEFs_GapMethod_withLonglivedLEFs_run'
folders = [foldername for foldername in os.listdir('Data/') if base_folder_string in foldername]


base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir = folders[m]

    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    sep_numerical_sorted = [float(a) for a in sep_list_sorted]
    proc_list = list(set([stats_file[i].split('proc')[1].split('_superportion')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_sorted = [float(a) for a in proc_list_sorted]
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [float(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    dsb_numerical_sorted = [float(a) for a in dsb_list_sorted ]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [float(a) for a in superloading_list_sorted]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]


    if m == 0:
        SuperConstrainingPortion = np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        SuperGapbridgingPortion =  np.zeros((len(folders),len(superportion_list_sorted),len(sep_proc_list)))
        
    for i in range(len(superportion_list_sorted)):
        for j in range(len(sep_proc_list)):
            proc = float(sep_proc_list[j].split('_proc')[1])
            if proc == 62.5:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(proc*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]
            else:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[i] + '_superproc' + str(int(proc*20)) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[0]
        #             print(par_combo)
            filename = 'Data/'+ data_dir + '/LEFidentity_' + par_combo + '.npy'

            if path.exists(filename):
                LEF_identity = np.load(filename)
                success_index = np.nonzero(np.sum(LEF_identity,axis=0))
                SuperConstrainingPortion[m,i,j] = np.count_nonzero(LEF_identity[0,success_index])/len(LEF_identity[0,success_index][0])*100
                SuperGapbridgingPortion[m,i,j] = np.mean((LEF_identity[2,success_index]+LEF_identity[4,success_index])/np.sum(LEF_identity[1:,success_index],axis=0))*100



colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))

# processivity

for i in range(len(sep_proc_list)):
    axs[0].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.mean(SuperConstrainingPortion,axis=0)[:,i]),yerr =stats.sem(SuperConstrainingPortion,axis=0)[:,i],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)
    
# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    

axs[0].set_ylim(0,100)
axs[0].set_ylabel('% Long-lived constraining LEFs')
axs[0].set_xlabel('% Long-lived LEFs')
axs[0].set_xticks(superportion_numerical_sorted[0:])
axs[0].tick_params(direction='out', length=8, width=2)
    
for i in range(len(sep_proc_list)):
    axs[1].errorbar(np.asarray(superportion_numerical_sorted), np.transpose(np.mean(SuperGapbridgingPortion,axis=0)[:,i]),yerr =stats.sem(SuperGapbridgingPortion,axis=0)[:,i],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)
axs[1].set_ylim(0,100)
axs[1].set_ylabel('% Long-lived gap-bridging LEFs')
axs[1].set_xlabel('% Long-lived LEFs')
axs[1].tick_params(direction='out', length=8, width=2)
axs[1].set_xticks(superportion_numerical_sorted[0:])
axs[1].tick_params(direction='out', length=8, width=2)

for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+ sep_proc.split('_proc')[1] for sep_proc in sep_proc_list]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')

plt.savefig('Figures/'+'LEF_identity.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()